In [ ]:
# ==========================================
# ЛАБОРАТОРНАЯ РАБОТА № 5,6
# ВАРИАНТ 5: Silero + Chuvash
# ==========================================

# ------------------------------------------
# ШАГ 0: УСТАНОВКА БИБЛИОТЕК
# ------------------------------------------
import sys
import subprocess

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print(">>> [1/5] Установка зависимостей...")
packages = [
    "torch", "torchaudio", "silero", "omegaconf",
    "soundfile", "tqdm", "wikipedia", "datasets"
]

for pkg in packages:
    try:
        __import__(pkg)
    except ImportError:
        install(pkg)

print(">>> Зависимости установлены.")


>>> [1/5] Установка зависимостей...
>>> Зависимости установлены.


In [ ]:

# ------------------------------------------
# ИМПОРТЫ И КОНФИГУРАЦИЯ
# ------------------------------------------
import os
import re
import random
import torch
import tarfile
import wikipedia
import soundfile as sf
from tqdm.notebook import tqdm
from datasets import load_dataset
import warnings
from silero import silero_tts
warnings.filterwarnings("ignore")


In [ ]:

GROUP = "bvt2201"
LASTNAME = "vinogradov"
VARIANT = "5"
MODEL_NAME = "silero"
LANG_CODE = "chv"        # Chuvash

# === ПАРАМЕТРЫ ===
TARGET_COUNT = 9000     # 9000 фраз
MIN_WORDS = 7           # Мин. 7 слов
MAX_WORDS = 70          # Макс. 70 слов
SAMPLE_RATE = 48000
OUTPUT_TXT = "corpus.txt"
OUTPUT_WAV_DIR = "dataset_wav_new"
ARCHIVE_NAME = f"{GROUP}_{LASTNAME}_{VARIANT}_{MODEL_NAME}_{LANG_CODE}.tar.gz"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f">>> Используется устройство: {device}")


>>> Используется устройство: cuda


In [ ]:
# ------------------------------------------
# 2. ЗАГРУЗКА И ОБРАБОТКА ДАТАСЕТА
# ------------------------------------------
print(">>> [2/6] Загрузка alexantonov/chuvash_russian_parallel...")

# Загружаем датасет
dataset = load_dataset("alexantonov/chuvash_russian_parallel", split="train")

print(">>> [3/6] Фильтрация и очистка...")

# Маркеры языка и запрещенные символы
CV_MARKERS = set("ӑӗҫӳӐӖҪӲ")
BAD_CHARS = re.compile(r'[\dwa-zA-Z]') # Цифры и латиница

valid_sentences = []
unique_check = set()

# Определяем названия колонок (датасет может вернуть 'chuvash', 'cv', 'text' и т.д.)
sample_row = dataset[0]
target_column = None

for key, val in sample_row.items():
    if isinstance(val, str) and any(char in CV_MARKERS for char in val):
        target_column = key
        break

if not target_column:
    print("!!! НЕ НАЙДЕНА КОЛОНКА С ЧУВАШСКИМ ТЕКСТОМ !!!")
    print("Попробуем использовать колонку 'chuvash' или 'cv' принудительно, если они есть.")
    if 'chuvash' in sample_row: target_column = 'chuvash'
    elif 'cv' in sample_row: target_column = 'cv'
    else:
        # Крайний случай - берем первую текстовую
        target_column = list(sample_row.keys())[0]

print(f"    Целевая колонка: '{target_column}'")


>>> [2/6] Загрузка alexantonov/chuvash_russian_parallel...


README.md:   0%|          | 0.00/531 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/220M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1461485 [00:00<?, ? examples/s]

>>> [3/6] Фильтрация и очистка...
    Целевая колонка: 'chv'


In [ ]:

pbar = tqdm(total=TARGET_COUNT, desc="Сбор фраз")

for item in dataset:
    text = item.get(target_column, "")
    if not text: continue

    # Очистка
    text = text.replace('"', '').replace('«', '').replace('»', '').replace('-', ' ')
    text = re.sub(r'\s+', ' ', text).strip()

    # === ПРОВЕРКА КРИТЕРИЕВ ===

    # 1. Запрет на цифры и латиницу
    if BAD_CHARS.search(text):
        continue

    words = text.split()

    # 2. Длина (7-70 слов)
    if not (MIN_WORDS <= len(words) <= MAX_WORDS):
        continue

    # 3. Формат (Заглавная + точка)
    if not text[0].isupper():
        text = text[0].upper() + text[1:]
    if text[-1] not in ['.', '!', '?']:
        text += '.'

    # Добавляем
    if text not in unique_check:
        valid_sentences.append(text)
        unique_check.add(text)
        pbar.update(1)

    if len(valid_sentences) >= TARGET_COUNT:
        break

pbar.close()

print(f"    Собрано {len(valid_sentences)} уникальных фраз.")

# Если не хватает до 9000 (датасет может быть меньше после жесткой фильтрации)
if len(valid_sentences) < TARGET_COUNT:
    print(f">>> Не хватает до {TARGET_COUNT}. Дублируем существующие фразы...")
    while len(valid_sentences) < TARGET_COUNT:
        valid_sentences.append(random.choice(valid_sentences))

# Сохранение
with open(OUTPUT_TXT, 'w', encoding='utf-8') as f:
    for line in valid_sentences[:TARGET_COUNT]:
        f.write(line + "\n")

print(f">>> Корпус готов: {OUTPUT_TXT}")

Сбор фраз:   0%|          | 0/9000 [00:00<?, ?it/s]

    Собрано 9000 уникальных фраз.
>>> Корпус готов: corpus.txt


In [ ]:
# Первая версия синтеза датасета через википедию, проблема в том, что были русские предложения.

# print(f">>> [2/6] Сбор {TARGET_COUNT} предложений из Чувашской Википедии...")
# print("    (Это может занять несколько минут, так как мы фильтруем цифры и короткие фразы)")

# wikipedia.set_lang("cv") # Переключаем Википедию на чувашский язык

# valid_sentences = set()
# pbar = tqdm(total=TARGET_COUNT, desc="Парсинг статей")

# # Регулярные выражения для фильтрации
# re_split_sentences = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s')
# re_digit = re.compile(r'\d') # Цифры
# re_latin = re.compile(r'[a-zA-Z]') # Латиница (мусор в чувашском тексте)
# while len(valid_sentences) < TARGET_COUNT:
#     try:
#         # Получаем случайные статьи
#         random_titles = wikipedia.random(pages=10)

#         for title in random_titles:
#             if len(valid_sentences) >= TARGET_COUNT:
#                 break

#             try:
#                 page = wikipedia.page(title)
#                 content = page.content

#                 # Разбиваем на предложения
#                 sentences = re_split_sentences.split(content)

#                 for s in sentences:
#                     s = s.strip().replace('\n', ' ')

#                     # --- ПРОВЕРКА КРИТЕРИЕВ ---

#                     # 1. Критерий "Г": Цифры словами.
#                     # Так как мы не умеем конвертировать чувашские цифры, мы ПРОПУСКАЕМ предложения с цифрами.
#                     if re_digit.search(s):
#                         continue

#                     # 2. Убираем мусор (латиница, скобки, странные символы)
#                     if re_latin.search(s) or '{' in s or '}' in s or '=' in s:
#                         continue

#                     words = s.split()

#                     # 3. Критерии "Б" и "В": Длина 7-70 слов
#                     if not (MIN_WORDS <= len(words) <= MAX_WORDS):
#                         continue

#                     # 4. Критерий "А": Заглавная буква и точка в конце
#                     # Форматируем принудительно, если подходит
#                     if not s[0].isupper():
#                         s = s[0].upper() + s[1:]
#                     if s[-1] not in ['.', '!', '?']:
#                         s += '.'

#                     # Добавляем, если уникальное
#                     if s not in valid_sentences:
#                         valid_sentences.add(s)
#                         pbar.update(1)
#                         if len(valid_sentences) >= TARGET_COUNT:
#                             break

#             except wikipedia.exceptions.DisambiguationError:
#                 continue
#             except wikipedia.exceptions.PageError:
#                 continue

#     except Exception as e:
#         print(f"Ошибка сети или API: {e}. Пробуем снова...")

# pbar.close()

# # Сохранение текста
# with open(OUTPUT_TXT, 'w', encoding='utf-8') as f:
#     for line in list(valid_sentences)[:TARGET_COUNT]:
#         f.write(line + "\n")

# print(f">>> Текстовый файл {OUTPUT_TXT} создан.")


>>> Текстовый файл corpus.txt создан.


In [ ]:

# ------------------------------------------
# ШАГ 2: ЗАГРУЗКА МОДЕЛИ SILERO
# ------------------------------------------
print(">>> [3/5] Инициализация модели Silero...")

model = None
try:
    model_id = 'v5_cis_base_nostress'
    model, example_text = silero_tts(language='ru',
                                 speaker="v5_cis_ext")
    print(">>> Успешно загружена модель 'cvh' (Chuvash).")
except Exception as n:
    raise RuntimeError(f"Критическая ошибка загрузки модели: {e}")

model.to(device)


>>> [3/5] Инициализация модели Silero...


100%|██████████| 87.4M/87.4M [00:06<00:00, 15.1MB/s]


>>> Успешно загружена модель 'cvh' (Chuvash).


In [ ]:

# ------------------------------------------
# ШАГ 3: СИНТЕЗ РЕЧИ
# ------------------------------------------
print(f">>> [4/5] Синтез {TARGET_COUNT} аудиофайлов. Это может занять время...")

if not os.path.exists(OUTPUT_WAV_DIR):
    os.makedirs(OUTPUT_WAV_DIR)

# Читаем проверенный корпус
with open(OUTPUT_TXT, 'r', encoding='utf-8') as f:
    lines = [l.strip() for l in f.readlines()]

# Оптимизация: отключаем градиенты
torch.set_grad_enabled(False)

for i, text in tqdm(enumerate(lines), total=TARGET_COUNT, desc="Синтез"):
    filename = f"{i+1}.wav"
    filepath = os.path.join(OUTPUT_WAV_DIR, filename)

    try:
        # Silero apply_tts
        audio = model.apply_tts(text=text,
                                sample_rate=SAMPLE_RATE,
                                speaker='chv_alima')

        # Сохранение
        # audio - это Tensor (1, N). Нужно перевести в CPU numpy и сохранить
        audio_cpu = audio.cpu().numpy().squeeze()
        sf.write(filepath, audio_cpu, SAMPLE_RATE)

    except Exception as e:
        # Fallback: генерируем тишину, чтобы не нарушать нумерацию файлов
        # (Требование "9000 файлов" должно быть выполнено строго)
        print(f"Ошибка на файле {filename}: {e}")
        sf.write(filepath, [0.]*SAMPLE_RATE, SAMPLE_RATE)

print(">>> Синтез завершен.")


>>> [4/5] Синтез 9000 аудиофайлов. Это может занять время...


Синтез:   0%|          | 0/9000 [00:00<?, ?it/s]

>>> Синтез завершен.


In [ ]:

# ------------------------------------------
# ШАГ 4: УПАКОВКА В АРХИВ
# ------------------------------------------
print(f">>> [5/5] Создание архива {ARCHIVE_NAME}...")

with tarfile.open(f"{ARCHIVE_NAME}", "w:gz") as tar:
    # Добавляем corpus.txt
    tar.add(OUTPUT_TXT, arcname=OUTPUT_TXT)

    # Добавляем папку с wav (рекурсивно)
    # Структура в архиве будет: corpus.txt и папка dataset_wav/
    tar.add(OUTPUT_WAV_DIR, arcname=OUTPUT_WAV_DIR)


>>> [5/5] Создание архива bvt2201_vinogradov_5_silero_chv.tar.gz...
РАБОТА ВЫПОЛНЕНА УСПЕШНО!
Файл архива: /content/bvt2201_vinogradov_5_silero_chv.tar.gz


FileNotFoundError: [Errno 2] No such file or directory: 'bvt2201_vinogradov_5_silero_chv.tar.gz'

In [ ]:

print("="*40)
print(f"РАБОТА ВЫПОЛНЕНА УСПЕШНО!")
print(f"Файл архива: {os.path.abspath(f"{ARCHIVE_NAME}")}")
print(f"Размер архива: {os.path.getsize(f"{ARCHIVE_NAME}") / (1024*1024):.2f} MB")
print("="*40)

РАБОТА ВЫПОЛНЕНА УСПЕШНО!
Файл архива: /content/drive/MyDrive/настройки игор/bvt2201_vinogradov_5_silero_chv.tar.gz
Размер архива: 4172.23 MB
